In [82]:
import pandas as pd
import glob, os

In [83]:
# df = pd.read_csv('/Users/biraj/Desktop/Github/flight-prediction-api/datasets/flights_2021_all.csv')
# df_w = pd.read_csv('/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_data/ATL_2021.csv',parse_dates=['date'])

df = pd.read_csv('../datasets/flights_2021_all.csv')
df_w = pd.read_csv('../datasets/weather_data/ATL_2021.csv',parse_dates=['date'])

In [84]:
df.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15
0,0,2021,1,1,5,2021-01-05,11298,DFW,11433,DTW,1625,1623.0,0.0,0.0,2005,2000.0,0.0,0.0
1,1,2021,1,1,6,2021-01-06,11298,DFW,11433,DTW,1625,1628.0,3.0,0.0,2005,2005.0,0.0,0.0
2,2,2021,1,1,12,2021-01-12,11298,DFW,11433,DTW,1625,1620.0,0.0,0.0,2005,2000.0,0.0,0.0
3,3,2021,1,1,13,2021-01-13,11298,DFW,11433,DTW,1625,1621.0,0.0,0.0,2005,1954.0,0.0,0.0
4,4,2021,1,1,19,2021-01-19,11298,DFW,11433,DTW,1625,1625.0,0.0,0.0,2005,1944.0,0.0,0.0


# Datetime Conversion

- A new column <code>CombinedDateTime</code> by combining <code>FlightDate</code> column and   <code>CRSDepTime</code> column.
- Scale the column <code>ScaledCRSDepTime</code> to the nearest hour.

In [85]:
df['CRSDepTime'] = df['CRSDepTime'].astype('str')

In [86]:
df['CombinedDateTime'] = pd.to_datetime(df['FlightDate'] + ' ' + df['CRSDepTime'].str.zfill(4), format='%Y-%m-%d %H%M')

In [87]:
df['ScaledCRSDepTime'] = pd.to_datetime(df['CombinedDateTime']).dt.round('H')


In [88]:
df.sample(10)

,Unnamed: 0,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,CombinedDateTime,ScaledCRSDepTime
308925,308925,2021,3,8,26,2021-08-26,13930,ORD,14107,PHX,2000,2018.0,18.0,1.0,2141,2148.0,7.0,0.0,2021-08-26 20:00:00,2021-08-26 20:00:00
136581,136581,2021,2,4,1,2021-04-01,13930,ORD,11298,DFW,1000,957.0,0.0,0.0,1248,1159.0,0.0,0.0,2021-04-01 10:00:00,2021-04-01 10:00:00
177661,177661,2021,2,5,29,2021-05-29,12266,IAH,11057,CLT,1200,1158.0,0.0,0.0,1530,1527.0,0.0,0.0,2021-05-29 12:00:00,2021-05-29 12:00:00
29568,29568,2021,1,1,19,2021-01-19,14107,PHX,13232,MDW,1850,1901.0,11.0,0.0,2305,2300.0,0.0,0.0,2021-01-19 18:50:00,2021-01-19 19:00:00
53517,53517,2021,1,2,4,2021-02-04,11057,CLT,10821,BWI,1740,1737.0,0.0,0.0,1905,1905.0,0.0,0.0,2021-02-04 17:40:00,2021-02-04 18:00:00
312411,312411,2021,3,8,18,2021-08-18,11292,DEN,14107,PHX,2110,2112.0,2.0,0.0,2200,2153.0,0.0,0.0,2021-08-18 21:10:00,2021-08-18 21:00:00
49314,49314,2021,1,2,2,2021-02-02,11292,DEN,11057,CLT,1148,1140.0,0.0,0.0,1658,1637.0,0.0,0.0,2021-02-02 11:48:00,2021-02-02 12:00:00
143990,143990,2021,2,4,11,2021-04-11,12266,IAH,12892,LAX,620,623.0,3.0,0.0,748,803.0,15.0,1.0,2021-04-11 06:20:00,2021-04-11 06:00:00
136793,136793,2021,2,4,1,2021-04-01,10821,BWI,13930,ORD,1120,1105.0,0.0,0.0,1236,1216.0,0.0,0.0,2021-04-01 11:20:00,2021-04-01 11:00:00
301928,301928,2021,3,8,22,2021-08-22,11292,DEN,14869,SLC,850,851.0,1.0,0.0,1018,1031.0,13.0,0.0,2021-08-22 08:50:00,2021-08-22 09:00:00


# Weather data

In [89]:
PATH_TO_FILE = '/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_data'
os.chdir(PATH_TO_FILE)
airport_list = []
for file in glob.glob("*.csv"):
    airport_list.append(file)
print(airport_list)

['SEA_2021.csv', 'SLC_2021.csv', 'SFO_2021.csv', 'ORD_2021.csv', 'ATL_2021.csv', 'CLT_2021.csv', 'DEN_2021.csv', 'LAX_2021.csv', 'PHX_2021.csv', 'DTW_2021.csv', 'PHL_2021.csv', 'BWI_2021.csv', 'IAH_2021.csv', 'MDW_2021.csv', 'DFW_2021.csv']


In [90]:
df_w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  8760 non-null   datetime64[ns]
 1   temperature_2m        8760 non-null   float64       
 2   relative_humidity_2m  8760 non-null   float64       
 3   dew_point_2m          8760 non-null   float64       
 4   precipitation         8760 non-null   float64       
 5   rain                  8760 non-null   float64       
 6   snowfall              8760 non-null   float64       
 7   weather_code          8760 non-null   float64       
 8   surface_pressure      8760 non-null   float64       
 9   cloud_cover           8760 non-null   float64       
 10  wind_speed_10m        8760 non-null   float64       
 11  wind_direction_10m    8760 non-null   float64       
dtypes: datetime64[ns](1), float64(11)
memory usage: 821.4 KB


In [91]:
df_w.head()

,date,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snowfall,weather_code,surface_pressure,cloud_cover,wind_speed_10m,wind_direction_10m
0,2021-01-01 00:00:00,12.3025,98.369354,12.052500,0.0,0.0,0.0,3.0,983.43170,100.0,12.727922,98.13002
1,2021-01-01 01:00:00,12.4525,98.695120,12.252501,0.0,0.0,0.0,3.0,983.64320,100.0,14.332341,101.59215
2,2021-01-01 02:00:00,12.6525,99.021450,12.502501,0.0,0.0,0.0,3.0,984.15045,100.0,14.799459,108.43504
3,2021-01-01 03:00:00,12.9025,99.023340,12.752501,0.0,0.0,0.0,3.0,984.18110,100.0,15.937878,108.43504
4,2021-01-01 04:00:00,13.1525,99.349230,13.052500,0.1,0.1,0.0,51.0,983.92250,100.0,17.208603,105.78082


In [92]:
df_w.rename(columns={'date':'ScaledCRSDepTime'},inplace=True)

In [93]:
df_w.head()

,ScaledCRSDepTime,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,snowfall,weather_code,surface_pressure,cloud_cover,wind_speed_10m,wind_direction_10m
0,2021-01-01 00:00:00,12.3025,98.369354,12.052500,0.0,0.0,0.0,3.0,983.43170,100.0,12.727922,98.13002
1,2021-01-01 01:00:00,12.4525,98.695120,12.252501,0.0,0.0,0.0,3.0,983.64320,100.0,14.332341,101.59215
2,2021-01-01 02:00:00,12.6525,99.021450,12.502501,0.0,0.0,0.0,3.0,984.15045,100.0,14.799459,108.43504
3,2021-01-01 03:00:00,12.9025,99.023340,12.752501,0.0,0.0,0.0,3.0,984.18110,100.0,15.937878,108.43504
4,2021-01-01 04:00:00,13.1525,99.349230,13.052500,0.1,0.1,0.0,51.0,983.92250,100.0,17.208603,105.78082


# Combine two datasets

In [94]:
df.columns 

Index(['Unnamed: 0', 'Year', 'Quarter', 'Month', 'DayofMonth', 'FlightDate',
       'OriginAirportID', 'Origin', 'DestAirportID', 'Dest', 'CRSDepTime',
       'DepTime', 'DepDelayMinutes', 'DepDel15', 'CRSArrTime', 'ArrTime',
       'ArrDelayMinutes', 'ArrDel15', 'CombinedDateTime', 'ScaledCRSDepTime'],
      dtype='object')

,Unnamed: 0,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,CombinedDateTime,ScaledCRSDepTime
0,0,2021,1,1,5,2021-01-05,11298,DFW,11433,DTW,1625,1623.0,0.0,0.0,2005,2000.0,0.0,0.0,2021-01-05 16:25:00,2021-01-05 16:00:00
1,1,2021,1,1,6,2021-01-06,11298,DFW,11433,DTW,1625,1628.0,3.0,0.0,2005,2005.0,0.0,0.0,2021-01-06 16:25:00,2021-01-06 16:00:00
2,2,2021,1,1,12,2021-01-12,11298,DFW,11433,DTW,1625,1620.0,0.0,0.0,2005,2000.0,0.0,0.0,2021-01-12 16:25:00,2021-01-12 16:00:00
3,3,2021,1,1,13,2021-01-13,11298,DFW,11433,DTW,1625,1621.0,0.0,0.0,2005,1954.0,0.0,0.0,2021-01-13 16:25:00,2021-01-13 16:00:00
4,4,2021,1,1,19,2021-01-19,11298,DFW,11433,DTW,1625,1625.0,0.0,0.0,2005,1944.0,0.0,0.0,2021-01-19 16:25:00,2021-01-19 16:00:00


In [96]:
csv_folder = '/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_data/'
for file_name in os.listdir(csv_folder):
    if file_name.endswith('.csv') and file_name[:3] in df['Origin'].unique():
        # Extract origin from file name
        origin = file_name[:3]

        # Read CSV file into a DataFrame
        file_path = os.path.join(csv_folder, file_name)
        file_df = pd.read_csv(file_path,parse_dates=['date'])
        file_df.rename(columns={'date':'ScaledCRSDepTime'},inplace=True)

        # Merge based on Origin and ScaledCRSDepTime, and add only the 10 columns
        merge_cols_file = ['ScaledCRSDepTime'] + [col for col in file_df.columns if col != 'ScaledCRSDepTime']
        df = pd.merge(df, file_df[merge_cols_file], 
                      left_on=['Origin', 'ScaledCRSDepTime'], 
                      right_on=[origin, 'ScaledCRSDepTime'], 
                      how='left', suffixes=('', f'_{origin}'))

KeyError: 'SEA'

In [ ]:
# csv_folder = '/Users/biraj/Desktop/Github/flight-prediction-api/datasets/weather_data'
# for file_name in os.listdir(csv_folder):
#     if file_name.endswith('.csv') and file_name[:3] in df['Origin'].unique():
#         # Extract origin from file name
#         origin = file_name[:3]

#         # Read CSV file into a DataFrame
#         file_path = os.path.join(csv_folder, file_name)
#         file_df = pd.read_csv(file_path,parse_dates=['date'])
#         file_df.rename(columns={'date':'ScaledCRSDepTime'},inplace=True)

#         # Merge based on Origin and ScaledCRSDepTime, and add only the 10 columns
#         merge_cols_file = ['ScaledCRSDepTime'] + [col for col in file_df.columns if col != 'ScaledCRSDepTime']
#         df = pd.merge(df, file_df[merge_cols_file], 
#                       left_on=['Origin', 'ScaledCRSDepTime'], 
#                       right_on=[origin, 'ScaledCRSDepTime'], 
#                       how='left', suffixes=('', f'_{origin}'))




In [ ]:
# for airport in airport_list:
#     filename = f"../weather_data/{airport}"
#     df_w = pd.read_csv(filename)
#     df_filter_by_origin = df[df['Origin']==airport[:3]]
#     pd.merge(df_filter_by_origin, df_w, on='ScaledCRSDepTime', how='left')

     